# A generational model of support for gun control

Allen Downey

[MIT License](https://en.wikipedia.org/wiki/MIT_License)

In [72]:
import pandas as pd
import numpy as np

import thinkstats2
import thinkplot
import utils

import matplotlib.pyplot as plt
import matplotlib

import seaborn as sns
sns.set(style='white', font_scale=1.0, context='talk')

from collections import Counter

import statsmodels.formula.api as smf
from statsmodels.discrete.discrete_model import MNLogit
from statsmodels.discrete.discrete_model import Logit

In [73]:
def read_samples(iters=101):
    """Read samples.
    
    iters: number of times to run
    """
    for i in range(iters):
        key = 'iter%d' % i
        sample = pd.read_hdf('iterations2021.hdf', key)
        yield sample

In [74]:
for sample in read_samples(1):
    pass

### Run logistic models

In [75]:
sample.shape

(45546, 59)

In [76]:
sample.columns

Index(['age', 'cohort', 'educ', 'gun', 'gunlaw', 'hispanic', 'income',
       'natcrime', 'owngun', 'polviews', 'race', 'realinc', 'sex', 'srcbelt',
       'wtssall', 'year', 'cohort5', 'cohort10', 'year8', 'year4', 'age10',
       'age5', 'age3', 'twenties', 'thirties', 'forties', 'fifties', 'sixties',
       'seventies', 'eighties', 'nineties', 'favor', 'gunhome', 'threatened',
       'spendcrime', 'topincome', 'lowincome', 'liberal', 'moderate',
       'conservative', 'female', 'ishisp', 'black', 'otherrace', 'urban',
       'suburban', 'rural', 'college', 'lowrealinc', 'highrealinc', 'ones',
       'c', 'a', 'y', 'c2', 'a2', 'y2', 'y3', 'ac'],
      dtype='object')

In [77]:
# not including Hispanic, due to too much missing data

varnames = ['nineties', 'eighties', 'seventies', 'fifties', 'forties', 'thirties', 'twenties',
            'female', 'black', 'otherrace', 'conservative', 'liberal', 'lowrealinc', 'highrealinc',
            'college', 'urban', 'rural']

all_varnames = varnames + ['y', 'y2', 'y3', 'favor']

In [78]:
def copy_nan(df, varname, newvar):
    """Put a NaN into newvar in any place where varname is Nan.
    
    df: DataFrame
    varname: string old var name
    newvar: string new var name
    """
    df.loc[df[varname].isnull(), newvar] = np.nan

In [79]:
def make_boolean(df, varname, values, newvar):
    """Make a boolean variable.
    
    df: DataFrame
    varname: name of base variable
    values: sequence of values for varname
    newvar: name of new variable (recode)
    """
    #assert numnull(df[varname]) == 0
    df[newvar] = df[varname].isin(values).astype(float)
    copy_nan(df, varname, newvar)

In [80]:
def make_booleans(df):
    df['cohort10'] = utils.RoundIntoBins(df, 'cohort', 10)
    make_boolean(df, 'cohort10', [1920], 'twenties')
    make_boolean(df, 'cohort10', [1930], 'thirties')
    make_boolean(df, 'cohort10', [1940], 'forties')
    make_boolean(df, 'cohort10', [1950], 'fifties')
    make_boolean(df, 'cohort10', [1960], 'sixties')
    make_boolean(df, 'cohort10', [1970], 'seventies')
    make_boolean(df, 'cohort10', [1980], 'eighties')
    make_boolean(df, 'cohort10', [1990], 'nineties')
    make_boolean(df, 'gunlaw', [1.0], 'favor')
    make_boolean(df, 'owngun', [1.0], 'gunhome')
    make_boolean(df, 'gun', [1.0], 'threatened')
    make_boolean(df, 'natcrime', [1.0], 'spendcrime')
    make_boolean(df, 'income', [12], 'topincome')
    make_boolean(df, 'income', [1,2,3,4,5,6,7,8], 'lowincome')
    make_boolean(df, 'polviews', [1,2,3], 'liberal')
    make_boolean(df, 'polviews', [4], 'moderate')
    make_boolean(df, 'polviews', [6,7,8], 'conservative')
    make_boolean(df, 'sex', [2], 'female')
    make_boolean(df, 'hispanic', [2], 'ishisp')
    make_boolean(df, 'race', [2], 'black')
    make_boolean(df, 'race', [3], 'otherrace')
    make_boolean(df, 'srcbelt', [1,2,5], 'urban')
    make_boolean(df, 'srcbelt', [3,4], 'suburban')
    make_boolean(df, 'srcbelt', [6], 'rural')

    df['college'] = df['educ'] >= 13 
    copy_nan(df, 'educ', 'college')
    
    quantile25 = df['realinc'].quantile(0.25)
    df['lowrealinc'] = df['realinc'] <= quantile25 
    copy_nan(df, 'realinc', 'lowrealinc')

    quantile75 = df['realinc'].quantile(0.75)
    df['highrealinc'] = df['realinc'] >= quantile75 
    copy_nan(df, 'realinc', 'highrealinc')

In [81]:
def replace_invalid(df):
    df.gunlaw.replace([8, 9, 0], np.nan, inplace=True)
    df.owngun.replace([3, 8, 9, 0], np.nan, inplace=True)
    df.gun.replace([8, 9, 0], np.nan, inplace=True)
    df.natcrime.replace([8, 9, 0], np.nan, inplace=True)
    df.income.replace([0, 13, 98, 99], np.nan, inplace=True)
    df.realinc.replace([0], np.nan, inplace=True)                  # TODO: check this
    df.educ.replace([98,99], np.nan, inplace=True)
    df.polviews.replace([8, 9, 0], np.nan, inplace=True)
    df.age.replace([98, 99], np.nan, inplace=True)               # 89 means 89 or older
    df.hispanic.replace([98, 99, 0], np.nan, inplace=True)
    df.cohort.replace([9999], np.nan, inplace=True)

In [82]:
replace_invalid(sample)
sample.dropna(subset=['gunlaw', 'age', 'cohort'], inplace=True)
make_booleans(sample)
sample.shape

(45546, 59)

In [83]:
for varname in varnames:
    print(varname, sum(gss[varname].isnull()))

nineties 0
eighties 0
seventies 0
fifties 0
forties 0
thirties 0
twenties 0
female 12
black 47
otherrace 47
conservative 4753
liberal 4753
lowrealinc 4187
highrealinc 4187
college 104
urban 3668
rural 3668


Select just the columns we need

In [84]:
data = sample[all_varnames]
data.shape

(45546, 21)

In [85]:
data['favor'].dtype

dtype('float64')

In [86]:
formula = ('favor ~ y + y2 + y3 + nineties + eighties + seventies + fifties + forties + thirties + twenties + '
           'female + black + otherrace + conservative + liberal + lowrealinc + highrealinc + ' 
           'college + urban + rural')
model = smf.logit(formula, data=data).fit()

model.summary()

Optimization terminated successfully.
         Current function value: 0.528289
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  favor   No. Observations:                45546
Model:                          Logit   Df Residuals:                    45525
Method:                           MLE   Df Model:                           20
Date:                Sun, 28 Aug 2022   Pseudo R-squ.:                 0.05239
Time:                        19:49:14   Log-Likelihood:                -24061.
converged:                       True   LL-Null:                       -25392.
Covariance Type:            nonrobust   LLR p-value:                     0.000
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               1.1069      0.040     27.609      0.000       1.028       1.186
lowrealinc[T.True]      0.0085      0.028      0.301      0.763      -0.047       0.064
highrealinc[T.True]     0.0542      0.028      1.914      0.056      -0.001       0.110
college[T.True]         0.1420      0.024      5.833      0.000       0.094       0.190
y                       0.0170      0.002      9.685      0.000       0.014       0.020
y2                     -0.0001      0.000     -1.373      0.170      -0.000     5.9e-05
y3                  -3.509e-05   4.33e-06     -8.095      0.000   -4.36e-05   -2.66e-05
nineties               -0.4941      0.065     -7.644      0.000      -0.621      -0.367
eighties               -0.1553      0.054     -2.894      0.004      -0.260      -0.050
seventies              -0.1148      0.045     -2.533      0.011      -0.204      -0.026
fifties                -0.0265      0.034     -0.770      0.441      -0.094       0.041
forties                -0.0312      0.037     -0.849      0.396      -0.103       0.041
thirties               -0.1554      0.041     -3.819      0.000      -0.235      -0.076
twenties               -0.0743      0.044     -1.706      0.088      -0.160       0.011
female                  0.6593      0.023     29.097      0.000       0.615       0.704
black                   0.3886      0.036     10.833      0.000       0.318       0.459
otherrace               0.5792      0.054     10.658      0.000       0.473       0.686
conservative           -0.4443      0.029    -15.542      0.000      -0.500      -0.388
liberal                 0.3241      0.028     11.582      0.000       0.269       0.379
urban                  -0.2874      0.028    -10.430      0.000      -0.341      -0.233
rural                  -0.7002      0.037    -19.035      0.000      -0.772      -0.628
=======================================================================================
"""

Make a row for someone in 2016 with all booleans false.

In [87]:
def make_base():
    y = 2016 - 1990
    y2 = y**2
    y3 = y**3

    d = dict(y=y, y2=y2, y3=y3)
    for varname in varnames:
        d[varname] = 0

    return pd.Series(d)

base = make_base()
base

y                  26
y2                676
y3              17576
nineties            0
eighties            0
seventies           0
fifties             0
forties             0
thirties            0
twenties            0
female              0
black               0
otherrace           0
conservative        0
liberal             0
lowrealinc          0
highrealinc         0
college             0
urban               0
rural               0
dtype: int64

Make a DataFrame that contains one row for each case we want to consider.

In [88]:
def make_df_pred():
    def add_yminus(df, varname, offset):
        """Add a column with y minus an offset.
        
        df: DataFrame
        varname: string new var name
        offset: how much to shift y
        """
        df.loc[varname] = base
        df.loc[varname, 'y'] += offset
        df.loc[varname, 'y2'] = df.loc[varname, 'y']**2
        df.loc[varname, 'y3'] = df.loc[varname, 'y']**3
    
    base = make_base()
    df_pred = pd.DataFrame(columns=base.index, dtype=float)    
    df_pred.loc['base'] = base

    for varname in varnames:
        df_pred.loc[varname] = base
        df_pred.loc[varname, varname] = 1
    
    add_yminus(df_pred, 'yminus10', -10)
    add_yminus(df_pred, 'yminus20', -20)
    add_yminus(df_pred, 'yminus30', -30)
    add_yminus(df_pred, 'yminus40', -40)
    
    #df_pred.loc['lowest combo'] = base
    #low_vars = ['gunhome', 'nineties', 'rural', 
    #            'conservative', 'lowrealinc']
    #df_pred.loc['lowest combo', low_vars] = 1
    
    #df_pred.loc['highest combo'] = base
    #high_vars = ['female', 'otherrace', 'liberal', 
    #            'college', 'highrealinc']
    #df_pred.loc['highest combo', high_vars] = 1
    
    return df_pred
    
df_pred = make_df_pred()

In [89]:
pred = model.predict(df_pred) * 100
pred

base            69.831378
nineties        58.545912
eighties        66.463656
seventies       67.360211
fifties         69.270124
forties         69.170961
thirties        66.460942
twenties        68.243755
female          81.736015
black           77.344734
otherrace       80.509525
conservative    59.747730
liberal         76.194407
lowrealinc      70.009587
highrealinc     70.960231
college         72.735920
urban           63.456519
rural           53.471763
yminus10        76.853670
yminus20        76.789197
yminus30        73.864332
yminus40        71.870086
dtype: float64

In [90]:
pred - pred['base']

base             0.000000
nineties       -11.285466
eighties        -3.367722
seventies       -2.471167
fifties         -0.561254
forties         -0.660417
thirties        -3.370436
twenties        -1.587623
female          11.904638
black            7.513356
otherrace       10.678147
conservative   -10.083648
liberal          6.363029
lowrealinc       0.178209
highrealinc      1.128853
college          2.904542
urban           -6.374859
rural          -16.359615
yminus10         7.022292
yminus20         6.957819
yminus30         4.032955
yminus40         2.038709
dtype: float64

In [91]:
def make_result(pred):
    """Make a DataFrame with one row per case.
    
    pred: series of predictions
    """
    result = pd.DataFrame()
    result['pred'] = pred
    result['offset'] = pred - pred['base']
    return result

result = make_result(pred)

### Iterate

To estimate uncertainty due to random sampling and missing values, we have to iterate the procedure we just ran.

In [92]:
results = []
for sample in read_samples():
    replace_invalid(sample)
    sample.dropna(subset=['gunlaw', 'age', 'cohort'], inplace=True)
    make_booleans(sample)
    data = sample[all_varnames]

    model = smf.logit(formula, data=data).fit(disp=0)

    df_pred = make_df_pred()
    pred = model.predict(df_pred) * 100
    result = make_result(pred)
        
    results.append(result)

Process the results.

In [93]:
preds = [result.pred for result in results]
median, low, high = thinkstats2.PercentileRows(preds, [50, 5, 95])

estimates = pd.DataFrame(index=result.index)
estimates['low5'] = low
estimates['median'] = median
estimates['high95'] = high
estimates.round(0).astype(int)

,low5,median,high95
base,68,69,71
nineties,57,60,62
eighties,64,66,69
seventies,64,66,68
fifties,66,68,69
forties,66,68,70
thirties,65,67,69
twenties,66,68,70
female,81,82,83
black,75,76,78


In [94]:
def make_table(estimates):
    lines = estimates.round(1).to_html().split('\n')
    for line in lines:
        print(line)

Generate the table for the offsets.

In [66]:
preds = [result.offset for result in results]
median, low, high = thinkstats2.PercentileRows(preds, [50, 2.5, 97.5])

offsets = pd.DataFrame(index=result.index)
offsets['low2.5'] = low
offsets['median'] = median
offsets['high97.5'] = high
table = offsets.sort_values('median', ascending=False).round(0).astype(int)

In [67]:
output = pd.DataFrame(columns=['support', 'offset', '90% CI'])
for label, row in table.iterrows():
    low, median, high = row
    support = estimates.loc[label]['median'].round(0).astype(int)
    ci = '(%d, %d)' % (low, high)
    output.loc[label] = support, median, ci
    
output

,support,offset,90% CI
female,82,13,"(12, 14)"
otherrace,81,11,"(10, 13)"
yminus10,77,8,"(7, 8)"
yminus20,77,7,"(6, 8)"
black,76,7,"(6, 8)"
liberal,76,7,"(6, 8)"
yminus30,73,4,"(2, 5)"
college,73,4,"(3, 5)"
yminus40,71,2,"(0, 3)"
highrealinc,70,0,"(-1, 2)"


In [68]:
def make_table(offsets):
    lines = offsets.sort_values('median').round(1).to_html().split('\n')
    for line in lines:
        print(line)

In [69]:
output.loc[:'liberal']

,support,offset,90% CI
female,82,13,"(12, 14)"
otherrace,81,11,"(10, 13)"
yminus10,77,8,"(7, 8)"
yminus20,77,7,"(6, 8)"
black,76,7,"(6, 8)"
liberal,76,7,"(6, 8)"


In [70]:
output.loc['college': 'thirties']

,support,offset,90% CI
college,73,4,"(3, 5)"
yminus40,71,2,"(0, 3)"
highrealinc,70,0,"(-1, 2)"
base,69,0,"(0, 0)"
lowrealinc,69,0,"(-1, 1)"
forties,68,-1,"(-3, 1)"
fifties,67,-1,"(-3, 0)"
twenties,68,-1,"(-4, 0)"
eighties,67,-3,"(-5, -1)"
thirties,66,-3,"(-4, -1)"


In [71]:
output.loc['eighties': 'rural']

,support,offset,90% CI
eighties,67,-3,"(-5, -1)"
thirties,66,-3,"(-4, -1)"
seventies,66,-3,"(-5, -1)"
urban,63,-6,"(-7, -5)"
nineties,60,-9,"(-12, -6)"
conservative,59,-11,"(-12, -9)"
rural,54,-16,"(-18, -14)"
